In [3]:
pip install openai

  Using cached openai-1.93.0-py3-none-any.whl.metadata (29 kB)
  Using cached jiter-0.10.0-cp311-cp311-win_amd64.whl.metadata (5.3 kB)
Using cached openai-1.93.0-py3-none-any.whl (755 kB)
Using cached jiter-0.10.0-cp311-cp311-win_amd64.whl (209 kB)

   -------------------- ------------------- 1/2 [openai]
   -------------------- ------------------- 1/2 [openai]
   -------------------- ------------------- 1/2 [openai]
   -------------------- ------------------- 1/2 [openai]
   -------------------- ------------------- 1/2 [openai]
   -------------------- ------------------- 1/2 [openai]
   -------------------- ------------------- 1/2 [openai]
   -------------------- ------------------- 1/2 [openai]
   -------------------- ------------------- 1/2 [openai]
   -------------------- ------------------- 1/2 [openai]
   -------------------- ------------------- 1/2 [openai]
   -------------------- ------------------- 1/2 [openai]
   -------------------- ------------------- 1/2 [openai]
   ------

In [ ]:
# imports
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI



In [6]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [ ]:
openai = OpenAI()



# Let's make a quick call to a Frontier model to get started, as a preview!

In [ ]:
# To give you a preview -- calling OpenAI with these messages is this easy

message = "Hello, GPT! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello! It's great to hear from you! How can I assist you today?


## OK onwards with our first project

In [ ]:

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [10]:
# Let's try one out. Change the website and add print statements to follow along.

ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connec

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [11]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [12]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [13]:
print(user_prompt_for(ed))

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acqui

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```python
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]
```
To give you a preview, the next 2 cells make a rather simple call - we won't stretch the mighty GPT (yet!)

In [14]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [15]:
# To give you a preview -- calling OpenAI with system and user messages:

response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
print(response.choices[0].message.content)

Oh wow, you’re really throwing me some tough ones today, huh? Alright, here we go: 2 + 2 equals 4. Mind-blowing, right?


## And now let's build useful messages for GPT-4o-mini, using a function

In [16]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [17]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a

## Time to bring it together - the API for OpenAI is very simple!

In [18]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [19]:
summarize("https://edwarddonner.com")

"# Summary of Edward Donner's Website\n\nEdward Donner's website serves as a platform for his interests in coding, LLM (Large Language Models) experimentation, and electronic music production. As the co-founder and CTO of Nebula.io, he focuses on leveraging AI to improve how talent is sourced and managed, providing valuable tools for recruitment.\n\n## News and Announcements\n\n- **Connecting my courses – become an LLM expert and leader**: May 28, 2025\n- **2025 AI Executive Briefing**: May 18, 2025\n- **The Complete Agentic AI Engineering Course**: April 21, 2025\n- **LLM Workshop – Hands-on with Agents – resources**: January 23, 2025\n\nThe website emphasizes a community for those interested in LLM technology and offers insights into Edward's professional journey and expertise."

In [20]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [21]:
display_summary("https://edwarddonner.com")

# Summary of Edward Donner's Website

Edward Donner's website introduces him as a passionate coder and enthusiast of language models (LLMs). He is the co-founder and CTO of Nebula.io, a company focused on utilizing AI for talent discovery and management. Previously, he founded and led AI startup untapt, which was acquired in 2021. The platform developed by Nebula.io incorporates proprietary LLMs tailored for the talent sector and boasts patented matching models, receiving significant attention from media and customers alike.

## News and Announcements:
- **May 28, 2025**: Launch of courses aimed at becoming an expert and leader in LLMs.
- **May 18, 2025**: Announcement of a 2025 AI Executive Briefing.
- **April 21, 2025**: Introduction of "The Complete Agentic AI Engineering Course."
- **January 23, 2025**: Availability of resources for the "LLM Workshop – Hands-on with Agents."

The site emphasizes community engagement, with options to connect and subscribe to updates.

# Let's try more websites

Note that this will only work on websites that can be scraped using this simplistic approach.

Websites that are rendered with Javascript, like React apps, won't show up. See the community-contributions folder for a Selenium implementation that gets around this. You'll need to read up on installing Selenium (ask ChatGPT!)

Also Websites protected with CloudFront (and similar) may give 403 errors - many thanks Andy J for pointing this out.

But many websites will work just fine!

In [22]:
display_summary("https://cnn.com")

# CNN Website Summary

The CNN website provides comprehensive coverage of breaking news and various topics including U.S. and world news, politics, business, health, entertainment, science, climate, and sports. Key highlights from the latest news include:

- **Israel-Hamas Ceasefire Talks**: Ongoing discussions regarding a potential ceasefire in Gaza are being monitored.
- **Bryan Kohberger Case**: Kohberger admits to the murders of Idaho students, with a mixed verdict leading to his acquittal on serious charges.
- **North Korea's Troops**: Reports indicate North Korea may send up to 30,000 troops to support Russia amid ongoing conflicts.
- **Trump's Economic Moves**: Donald Trump announces a trade deal with Vietnam, promising to alter U.S. trade dynamics.
- **Tech Industry Layoffs**: Microsoft plans to lay off approximately 9,000 workers in its most significant cuts since 2023.
- **Health Findings**: New studies suggest that there is no safe level of processed meat consumption, raising health concerns.

CNN also features articles on lifestyle, global events, and science, while providing multimedia content such as videos and live TV. The site encourages user engagement through feedback and newsletters for personalized content delivery.

In [23]:
display_summary("https://anthropic.com")

# Anthropic Website Summary

Anthropic is focused on developing AI products, notably the Claude family of models, to promote safety and human well-being. Key features of the website include:

## Products
- **Claude Models**: The site highlights various AI models such as Claude Opus 4, Claude Sonnet 4, and Claude Haiku 3.5. These models are designed to handle complex tasks and coding.
- **APIs and Development**: Anthropic provides resources to build AI applications using the Claude API, along with developer documentation.

## Research and Safety
- **AI Safety**: Anthropic emphasizes responsible AI development through research, policy work, and product design, aiming to ensure technologies are beneficial to humanity.
- **Initiatives and Policies**: The site outlines their Responsible Scaling Policy and commitment to transparency and security in AI.

## Announcements
- **ISO 42001 Certification**: The latest announcement highlights that Anthropic has achieved ISO 42001 certification, indicating a recognition of the company's safety and compliance standards.

## Learning Resources
- **Anthropic Academy**: An educational platform where users can learn to utilize Claude effectively, featuring customer stories and engineering insights.

Overall, the website serves as a comprehensive resource for anyone interested in Claude AI models, their applications, and the safety principles guiding Anthropic's development strategies.

In [ ]:
# Step 1: Create your prompts

system_prompt = "something here"
user_prompt = """
    Lots of text
    Can be pasted here
"""

# Step 2: Make the messages list

messages = [] # fill this in

# Step 3: Call OpenAI

response =

# Step 4: print the result

print(